# Visualisierung im Jupyter-Notebook
(Achtung: Alles hier ist *work in progress*!)

## Pythontutor
[Pythontutor](https://pythontutor.com) ist ein geniales Tool, mit dem man Code-Ausführung und die dabei entstehenden Daten visualisieren kann! 

Um zu sehen, was es kann, unbedingt einige Beispiele anschauen. Diese findet man, wenn man auf [dieser Seite](https://pythontutor.com/visualize.html) den Link "Show code examples" anklickt (ist JS und kann deshalb hier nicht direkt verlinkt werden). Unter den Beispielen empfehle ich zB:
- intro (als Überblick über viele Feaures sowohl von Python als auch Python-Tutor)
- hello (zeigt Referenzen, auch mehrere auf dasselbe Objekt, und Funktionen, insb. deren Parameter und lokale Variablen)
- filter
- OOP2
- die Linked List Beispiele
- u.v.m. 

Will man "von Hand" die Ausführung eines Codebeispiels visualisieren, kopiert man diesen in das Textfeld auf dieser Seite: https://pythontutor.com/visualize.html#mode=edit

#### Pythontutor direkt im Notebook nutzen
Pythontutor kann man über eine Art API auch programmatisch in seinen eigenen Code einbauen, insb. in ein Jupyter Notebook. 

In diesem Notebook entwickele ich eine Funktion `visualisiere_code_zelle()`, die
- das Notebook nach einer Zelle mit einem bestimmten Stichwort durchsucht, zB "Aufgabe 2.13"
- den Inhalt dieser Zelle an PythonTutor schickt
- und den von PythonTutor erzeugten IFrame anzeigt

Die API von Pythontutor wird in [diesem Dokument](https://docs.google.com/document/d/1quk8gdvgzaYrZaOSiVzCia8tCXBFmuTk0awMw_-OTuE/edit?usp=sharing) beschrieben.

Hier ein "hartkodiertes" Beispiel:

In [1]:
%%html
<iframe width="800" height="400" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=liste%20%3D%20%5B1,%202,%203%5D%0Asumme%20%3D%200%0Afor%20zahl%20in%20liste%3A%0A%20%20%20%20summe%20%2B%3D%20zahl%0Aprint%28summe%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

Das Tag `<iframe>`ehält einen Bereich, der mit `code=...` beginnt. Darin findet sich der (für URLs angepasste) Python-Code.

Ein solches iframe wollen wir nun programmatisch erzeugen. Zuvor aber ein paar Notebook-Zellen mit Python-Beispielen, die wir nachher auslesen wollen:

### Beispiele
Hier ein paar Zellen mit einfachem Python-Code. Wichtig: In Zeile 1 steht eine Art "ID", nach der wir später suchen können.

In [2]:
# Aufgabe 1.03    Das ist natürlich nur ein willkürlich gewählter Name
liste = [1, 2, 3, 2, 4, 5, 6, 4]
summe = 0
bekannt = set()
for zahl in liste:
    if zahl in bekannt:
        continue
    summe += zahl
    bekannt.add(zahl)
print(summe)

21


## Visualierung
Wir schreiben zwei Funktionen:

1. `suche_zelle()` durchsucht die History des Jupyter Notebooks, d.h. die schon ausgeführten Zellen nach einem bestimmten Suchstring, und zwar *rückwärts*, so dass die zuletzt ausgeführte Zelle zuerst gefunden und zurückgegeben wird.
2. `visualisiere_code_zelle()` schickt den `suche_zelle()` gefundenen Code an Pythontutor und stellt das daraus erzeugte Iframe mit der Visualisierung des Codes dar.

In [3]:
# Code für Visualisierung von Notebook-Zellen mit Python Tutor
from typing import Optional, Generator
from urllib.parse import quote
from IPython.display import IFrame
import inspect

def suche_zelle(suchtext: str, suchtext_in_zeile_1=True, history=In, verboten=None) -> Optional[str]:
    def ist_treffer(text: str) -> bool:
        suchbereich = text.splitlines()[0] if suchtext_in_zeile_1 else text
        return suchtext in suchbereich and verboten not in suchbereich
    treffer: Generator[str, None, None] = (text for text in reversed(history[:-1]) if text and ist_treffer(text))
    neuester_treffer: Optional[str] = next(treffer, None)
    return neuester_treffer 


def visualisiere_code_zelle(suchtext: str, history=In, width="100%", height=500) -> Optional[IFrame]:
    # Codezelle finden
    name_dieser_funktion = inspect.currentframe().f_code.co_name  # verwendet, um die Zelle auszuschließen, in der der Visualisierungsaufruf steht
    code = suche_zelle(suchtext, history, verboten=name_dieser_funktion)
    if code is None:
        print(f'Keine Notebook-Zelle mit Suchtext "{suchtext}" gefunden. Wichtig: Vor Aufruf der Visualisierung muss die Zelle ausgeführt werden.')
        return None

    # Aufruf der PythonTutor-API vorbereiten
    quoted_code = quote(code)
    zeilen = len(code.splitlines())
    codeDivHeight = height - 150
    python_tutor_iframe = f"""https://pythontutor.com/iframe-embed.html#code={quoted_code}&codeDivHeight={codeDivHeight}&codeDivWidth=500&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false   """
    return IFrame(src=f'{python_tutor_iframe}', width=width, height=height)

In [4]:
# Beispiel 1.07  (inhaltlich recht sinnlos)
d = {}
d2 = d
d["Harry"] = "Potter"
d["Hermione"] = "Granger"

def voller_name(vorname):
    nachname = d[vorname]
    voll = f"{vorname} {nachname}"
    return voll

vorname = "Hermione"
vollstaendiger_name = voller_name(vorname)
print(f"Herzlich Willkommen, {vollstaendiger_name}")

Herzlich Willkommen, Hermione Granger


In [5]:
visualisiere_code_zelle("Aufgabe 1.07", In)

Keine Notebook-Zelle mit Suchtext "Aufgabe 1.07" gefunden. Wichtig: Vor Aufruf der Visualisierung muss die Zelle ausgeführt werden.


### Visualisierungsfunktion in externe Datei auslagern
Natürlich möchte man die "echten Inhalte" eines Notebooks von Hilfsfunktionen wie `visualisiere_code_zelle()` trennen. Dazu habe ich ein Python-Modul `unterricht.py` erstellt. Aus diese kann `visualisiere_code_zelle` importiert werden.

Problem: Aktuell muss an die externe Version von `visualisiere_code_zelle` als zusätzliches Argument die [Kommando-Historie `In`](https://ipython.org/ipython-doc/3/interactive/reference.html#input-caching-system) des Notebooks übergeben wird, denn diese wird nach dem Suchstring durchsucht - und ist leider aus einer externen Datei offensichtlich nicht mehr zugänglich.  Wahrscheinlich kann man das mit irgendeinem Hack lösen und dadurch den Aufruf für KuK einfacher gestalten - noch habe ich aber keinen gefunden...

In [6]:
from unterricht import visualisiere_code_zelle

visualisiere_code_zelle("1.03", In)